# Debugging

Objectives

今天课程的主题是调试。

首先，我们将探讨如何避免调试--编写代码的方法要么能完全避免调试，要么至少能在必须调试时让调试变得简单。

但有时你别无选择，只能进行调试--尤其是在将整个系统连接在一起时才发现的错误，或者是在系统部署后由用户报告的错误，在这种情况下，可能很难将错误定位到某个特定的模块。在这种情况下，我们可以建议采用系统化的策略来提高调试效率。

Andreas Zeller 所著的[Why Programs Fail ](https://mit.primo.exlibrisgroup.com/permalink/01MIT_INST/sjd9fk/cdi_safari_books_v2_9780123745156)是一本关于系统调试的好书。这本书中的很多内容都是受这本书的启发。

John Regehr 的 ["How to Debug"](https://blog.regehr.org/archives/199)也与此相关，它是嵌入式系统课程的讲义，比 6.102 更低级，但具有相同的系统调试一般原则。

最后，[ Debugging: The Nine Indispensable Rules for Finding Even the Most Elusive Software and Hardware Problems](https://mit.primo.exlibrisgroup.com/permalink/01MIT_INST/sjd9fk/cdi_skillsoft_books24x7_bks00002516)是一本可读性强、非常实用的指南，适用于从软件到硬件、从汽车到管道等各种技术环境下的调试。我们在本篇阅读中参考了 ["Nine Rules"](http://www.debuggingrules.com/Debugging_CH2.PDF#page=2)中的几条。

## First defense: make bugs impossible

抵御 bug 的最好办法就是在设计上让它们不可能出现。

我们已经讨论过的一种方法就是静态检查。静态检查通过在编译时捕获错误来消除许多错误。
| “Making mistakes is hard to avoid but not caring to prevent mistakes is unacceptable.”|
|---|
|Zeller, Why Programs Fail, Chapter 3|

我们在前面的阅读中也看到了一些动态检查的例子。例如，Python 通过动态捕获使数组溢出 bug 变得不可能。如果您试图使用超出 list 边界的索引，Python 会自动产生错误。JavaScript 则没有这么好，因为它会在读取错误时悄悄返回未定义。C 和 C++ 等老式语言的情况更糟，它们会默许超出固定大小数组边界的错误写入，从而导致错误和安全漏洞。

Immutability 是另一个防止漏洞的设计原则，它有两种类型：immutable types and unreassignable references（常量）。

字符串是一种不可变类型。在字符串上调用任何方法都不会改变字符串所代表的字符序列。字符串可以被传递和共享，而不用担心会被其他代码修改。

TypeScript 还提供了不可赋值引用：使用关键字 const 声明的变量，可以赋值一次，但永远不能再赋值。在声明局部变量时，尽可能使用 const 是一种很好的做法。与变量类型一样，这些声明也是重要的文档，对代码读者有用，编译器也会对其进行静态检查。

请看下面这个例子

```ts
const letters: Array<string> = ['a', 'e', 'i', 'o', 'u'];
```
letters变量被声明为 const，但它真的不变吗？下面哪些语句是非法的（被编译器静态捕获），哪些是允许的？

```ts
letters = ['x', 'y', 'z']; 
letters[0] = 'z';
```
您可以在下面的练习中找到答案。请注意 const 的含义！它只是使引用不可赋值，但引用指向的对象可能是可变的。

## Second defense: make bugs easy to find
静态检查、动态检查和immutability只能在一定程度上预防程序错误的出现，因此我们的下一个设计策略就是在程序错误不可避免地出现时，让它们更容易被发现。我们可以将错误定位到程序的一小部分，这样我们就不必在太广的范围内寻找错误的原因。如果将错误定位在单个方法或小模块上，只需研究程序文本就能发现错误。

我们已经讨论过 "fail fast"：越早发现问题（越接近其原因），就越容易修复。

让我们从一个简单的例子开始：
```ts
/**
 * @param x  requires x >= 0
 * @returns approximation to square root of x
 */
function sqrt(x: number): number { ... }

```

现在假设有人用一个负参数调用 sqrt。sqrt 的好的行为是什么？由于调用者没有满足 x 应该是非负值的要求，因此 sqrt 不再受其合同条款的约束，所以从技术上讲，它可以为所欲为：返回任意值，或进入无限循环，或关闭 CPU。然而，由于错误调用表明调用者存在错误，因此最有用的行为是尽早指出错误。为此，我们可以在运行时插入对前提条件的检查。下面是一种检查方法：

```ts
/**
 * @param x  requires x >= 0
 * @returns approximation to square root of x
 */
function sqrt(x: number): number { 
    if ( ! (x >= 0)) throw new Error("required x >= 0, but was: " + x);
    ...
}
```

当前提条件不满足时，该代码会抛出错误终止程序。调用者错误的影响不会传播。

检查前提条件是防御性编程的一个例子。真实的程序很少没有错误。Defensive programming提供了一种方法，即使你不知道错误在哪里，也能减轻错误的影响。

### Assertions
通常的做法是为这类防御检查定义一个函数，通常称为 assert。这种方法抽象出了断言失败时到底会发生什么。失败的断言可能会退出；可能会在日志文件中记录事件；可能会通过电子邮件向维护者发送报告。

JavaScript/TypeScript 没有内置断言，但我们将在 6.102 中使用 Node 的断言包。断言的最简单形式是使用布尔表达式，如果布尔表达式求值为 false，则抛出 AssertionError：
```ts
assert(x >= 0);
```
断言的另一个好处是，它记录了程序在这一点上的状态假设。对于阅读你代码的人来说，assert(x >= 0) 表示 "在这一点上，x >= 0 应该始终为真"。但与注释不同的是，断言是在运行时执行假设的可执行代码。

断言还可以包含字符串信息，在断言失败时打印出来。这可以用来向程序员提供有关失败原因的更多细节。例如

```ts
assert(x >= 0, "x is " + x);
```
If x === -1, then this assertion fails with the error message

> x is -1

配合堆栈跟踪，堆栈跟踪会告诉你断言是在代码中的哪个位置发现的，以及将程序带到这个位置的调用顺序。这些信息通常足以帮助您开始查找错误。

关于断言需要注意的一点是，在许多语言（如 Java）中，断言默认是关闭的。这意味着，如果你像往常一样运行程序，你的断言都不会被检查！语言设计者选择这样做的一个原因是，检查断言有时会对性能造成损失。例如，一个使用二进制搜索数组的函数要求对数组进行排序。然而，断言这一要求需要扫描整个数组，这就将本应在对数时间内运行的操作变成了需要线性时间的操作。在测试过程中，你应该愿意（渴望！）付出这样的代价，因为它能让调试变得更容易，但在程序发布给用户后就不一样了。不过，对于大多数应用程序来说，断言与代码的其他部分相比并不昂贵，而且它们在错误检查方面带来的好处也值得在性能上付出这么小的代价。

在默认关闭断言的语言中，请确保显式启用断言。我们在 6.102 中使用的 Node 断言库始终运行其断言语句，因此无需担心显式开启断言。

### What to assert

以下是您应该断言的一些事项：

**方法参数要求**，如上面的 sqrt。

**方法返回值要求**。这种断言有时称为自我检查。例如，sqrt 方法可能会对其结果进行平方，以检查它是否合理地接近 x：

```ts
function sqrt(x: number): number {
    assert(x >= 0);
    let r: number;
    ... // compute result r
    assert(Math.abs(r*r - x) < .0001);
    return r;
}
```
何时应编写运行时断言？在编写代码时，而不是事后。在编写代码时，您会考虑invariants。如果推迟编写断言，就不太可能做到这一点，而且很可能会遗漏一些重要的不变式。

### What not to assert
Runtime assertions不是免费的。它们会使代码变得杂乱无章，因此必须谨慎使用。避免使用琐碎的断言，就像避免使用无意义的注释一样。例如

```ts
// don't do this:
x = y + 1;
assert(x === y+1);
```

该断言不会发现代码中的错误。它只会发现编译器或 JavaScript 解释器中的错误，在没有充分理由怀疑编译器或 JavaScript 解释器之前，您应该信任它们。如果断言在本地上下文中是显而易见的，那么就不要使用它。

切勿使用断言来测试程序的外部条件，例如文件的存在性、网络的可用性或用户输入的正确性。断言会测试程序的内部状态，以确保其在规范范围内。当断言失败时，说明程序在某种意义上已经脱离了轨道，进入了设计时未考虑到的正常运行状态。因此，断言失败表明程序存在缺陷。外部故障并不是系统中的错误（它们甚至可能根本就不是错误！），你无法事先对程序做出任何修改来防止它们的发生。外部故障应使用异常来处理。

如前所述，许多断言机制都设计成仅在测试和调试期间执行断言，并在程序发布给用户时关闭断言。（重申一下，Node 断言的情况并非如此，它不能被禁用；但例如 Java 断言实际上默认是禁用的）。由于断言有时会被禁用，因此好的做法是确保程序的正确性不取决于断言表达式是否被执行。尤其是，断言表达式不应有副作用。例如，如果您想从列表中弹出一个元素，并断言在弹出之前列表是非空的，就不要这样写：

```ts
// don't do this:
assert(list.pop());
```
如果断言被禁用，整个表达式将被跳过，并且不会从列表中弹出任何项。可以这样写
```ts
const found = list.pop();
assert(found);
```

同样，如果条件语句或switch没有涵盖所有可能的情况，使用断言检查并阻止非法情况也是不错的做法。

```ts
switch (vowel) {
  case 'a':
  case 'e':
  case 'i':
  case 'o':
  case 'u': return 'A';
  default: assert.fail('should never get here');
}
```

### Incremental development

incremental development是将错误定位到程序极小部分的好方法。每次只编写一点程序，并在继续编写之前彻底测试这一点。这样，当你发现错误时，它更有可能出现在你刚刚编写的部分，而不是一大堆代码中的任何地方。

我们在测试课上介绍了两种有助于实现这一点的技术：

* 单元测试：当你孤立地测试一个模块时，你可以确信你发现的任何错误都在该单元中，或者可能在测试用例本身中。
* 回归测试：在大型系统中添加新功能时，尽可能频繁地运行回归测试套件。如果测试失败，错误很可能就在你刚刚修改的代码中。
我们还谈到了版本控制。经常添加、提交和推送不仅能让你的工作免于断电，也是保存工作中无错误代码的好方法。如果你经常添加、提交和推送工作代码，那么一旦出现错误，你就可以将你的修改与上次保存的工作版本进行比较，并根据差异进行调试。

### Modularity & encapsulation

你还可以通过更好的软件设计来定位错误。

Modularity是指将系统划分为多个组件或模块，每个组件或模块都可以独立于系统的其他部分进行设计、实施、测试、推理和重用。模块化系统的反面是单体系统--体积庞大，所有部件相互缠绕、相互依赖。

由一个个很长的函数组成的程序是单体的--更难理解，也更难隔离其中的错误。相比之下，分成小函数和小class的程序模块化程度更高。

封装是指在模块周围筑起围墙，使模块只对自己的内部行为负责，系统其他部分的错误不会破坏模块的完整性。

其中一种encapsulation是访问控制，即使用 public 和 private 来控制变量和方法的可见性和可访问性。任何代码都可以访问公共变量或方法（假设包含该变量或方法的类也是公共的）。私有变量或方法只能由同类中的代码访问。尽可能保持私有，尤其是变量的私有，可以提供封装性，因为它限制了可能无意中导致错误的代码。

另一种encapsulation来自变量的作用域。变量的作用域是程序中定义该变量的部分，即表达式和语句可以引用该变量。函数参数的作用域是函数的主体。局部变量的作用域从其声明到下一个结尾大括号。尽可能缩小变量的作用域，可以更容易地推理出程序中可能存在的错误。例如，假设有这样一个循环

```ts
for (i = 0; i < 100; ++i) {
    ...
    doSomeThings();
    ...
}
```
...然后你发现，这个循环一直在运行——但是i 从未达到 100。在某个地方，有人正在改变 i。如果 i 被声明为全局变量，就像下面这样：

```ts
let i: number;
...
for (i = 0; i < 100; ++i) {
    ...
    doSomeThings();
    ...
}
```
......那么它的作用域就是整个程序。它可能会在程序的任何地方发生变化：被 doSomeThings()、被 doSomeThings() 调用的其他方法、被运行完全不同代码的并发线程。但如果将 i 声明为作用域狭窄的局部变量，就像下面这样：

```ts
for (let i = 0; i < 100; ++i) {
    ...
    doSomeThings();
    ...
}
```
......唯一可以修改 i 的地方就是 for 语句中，事实上，只有在我们省略的......部分。你甚至不必考虑 doSomeThings()，因为 doSomeThings() 无法访问这个局部变量。

将变量的作用域最小化是实现 bug localization 的有效方法。以下是一些适用于 TypeScript 的规则：

* 始终在 for 循环初始化器中声明循环变量,不要在循环之前声明：
  ```ts
  let i: number;
  for (i = 0; i < 100; ++i) {
  ```
这使得变量的作用域是包含这段代码的整个外层大括号代码块的其余部分，你应该这样做：
```ts
for (let i = 0; i < 100; ++i) {
```
这使得 i 的作用域仅限于 for 循环。

* 一定要使用 const 或 let，千万不要使用 var。 const 或 let 变量的作用域是围绕它的最小的大括号块，而 var 声明的作用域是围绕它的整个函数（顺便提一下，Python 局部变量使用的也是同样的规则）。你会在网上看到很多使用 var 的旧 JavaScript 代码，请避免使用。现代 JavaScript/TypeScript 编程总是使用 const 或 let。

* 只有在第一次需要变量时，才在最内层的大括号块中声明变量。将变量声明放在包含所有需要使用该变量的表达式的最内层代码块中。不要在函数开始时声明所有变量--这会使变量的作用域变得不必要的大。

* 避免使用全局变量。使用全局变量是一个非常糟糕的主意，尤其是当程序变大时。使用全局变量作为在程序的多个部分之间共享某些数据的快捷方式可能很诱人，但千万不要上当。最好是直接将参数传递到需要它的代码中，而不是将它放在全局空间中，因为它可能在无意中被重新分配。




## Reproduce the bug
假设有人向你报告了你编写的软件中的一个错误（例如，在提交 pset 的 alpha 版本后，一个隐藏测试用例失败了）。这在软件工程生涯中是不可避免的。

首先要找到一个小型、可重复的测试用例来重现故障。如果错误是通过regression testing发现的，那你就走运了；你的测试套件中已经有了一个失败的测试用例。如果错误是由用户报告的，则可能需要花费一些精力来重现错误。对于图形用户界面和多线程程序来说，如果一个错误依赖于事件或线程执行的时间，那么它就很难持续重现。

尽管如此，您为使测试用例小巧且可重复而付出的任何努力都将得到回报，因为在您查找错误并开发修复方法时，您必须反复运行测试用例。此外，在成功修复漏洞后，您还需要在回归测试套件中保留该测试用例，这样漏洞就不会再出现了。一旦有了针对错误的测试用例，让这个测试正常工作就成了你的目标。

下面是一个例子。假设您编写了这样一个函数

```ts
/**
 * Find the most common word in a string.
 * @param text string containing zero or more words, where a word
 *     is a string of alphanumeric characters bounded by nonalphanumerics.
 * @returns a word that occurs maximally often in text, ignoring alphabetic case.
 */
function mostCommonWord(text: string): string {
    ...
}
```

用户将莎士比亚戏剧的全部文本传入你的方法（类似 mostCommonWord(allShakespearesPlaysConcatenated)），结果发现该方法返回的不是 "the "或 "a "等可预测的常见英文单词，而是一些意想不到的单词，可能是 "e"。

莎士比亚的戏剧有 100,000 行，包含 800,000 多个单词，因此这种输入调试起来会非常痛苦。如果首先将错误输入的大小减小到可以处理的程度，但仍能显示相同（或非常相似）的错误，调试就会变得更容易：

* 莎士比亚的前半部分是否显示了相同的错误？（然后继续减半）！（Binary search 通常是一种很好的技术。下文将详细介绍）。
* 一个剧本是否有相同的错误？
* 一个独白是否有相同的错误？
找到一个小测试用例后，用这个小测试用例找到并修复错误，然后回到原来的错误输入，确认你修复了同样的错误。



## Find the bug using the scientific 

要确定bug的位置和原因，可以使用scientific method：

1. Study the data。查看导致错误的测试输入，检查由此产生的错误结果、失败断言和堆栈跟踪。

2. Hypothesize。提出一个与所有数据一致的假设，说明错误可能出现在哪里，或者不可能出现在哪里。一开始最好提出一个笼统的假设。

3. Experiment。设计并运行一个实验来验证你的假设。实验一开始最好是观察--收集信息，但尽量少干扰系统。

4.Repeat。将从实验中收集到的数据与之前的知识相结合，提出新的假设。希望你已经排除了一些可能性，并缩小了错误的可能位置和原因范围。

并不是每个错误都需要这种深思熟虑的过程。有了好的快速故障设计，你就有希望在非常接近错误源头的地方发现异常，堆栈跟踪会引导你找到它，检查代码就会发现错误。那么什么时候需要运用科学方法呢？一个很好的经验法则是 10 分钟法则。如果你已经花了 10 分钟时间使用临时的、不系统的检查方法来查找错误，那么你就应该退一步，开始使用科学方法。

在这一转变过程中，您还应将调试过程从头脑中移出——头脑中关于您已经尝试过的以及从中了解到的，非常有限的工作记忆——并开始在纸上或笔记本电脑上做笔记。在调试过程的每一次迭代中，你都要记下来：

* Hypothesis。根据你目前所学到的知识，你下一个关于错误位置或原因的假设是什么？
* Experiment。通过验证或证伪假设，你准备尝试什么方法来揭示假设？
* Predictions。根据你的假设，你预计实验结果会怎样？
* Observations。做实验时实际发生了什么？
根据你过去在科学课上的经验，这些问题对你来说应该非常熟悉。在接下来的几节中，我们将看看这些问题在调试代码时会以何种形式出现。在调试过程中，有些假设和实验比其他假设和实验更好。

### 1. Study the data

异常的堆栈跟踪是一种重要的数据。练习阅读所获得的堆栈跟踪，因为它们会为你提供大量信息，让你知道错误可能出在哪里。

在典型的堆栈跟踪中，最新的调用在顶部，最旧的调用在底部。但堆栈顶部或底部的调用可能不是你编写的库代码。你自己的代码--也就是最有可能出现错误的地方--通常位于中间。不要因此而放弃。扫描堆栈跟踪，直到看到熟悉的东西，然后在自己的代码中找到它。

### 2. Hypothesize

程序抛出异常或产生错误答案的实际失败点并不一定就是错误所在。有漏洞的代码在最终失败之前，可能已经通过程序的良好部分传播了一些坏值。因此，你的假设应该是错误到底在哪里（或不在哪里），以及是什么原因导致了错误。

将程序视为数据流或算法中的步骤会有所帮助，可以尝试一次性排除程序的所有部分。让我们结合 mostCommonWord() 的示例来思考这个问题，它通过三个辅助方法得到了进一步的充实：

```ts
/**
 * Find the most common word in a string.
 * ...
 */
function mostCommonWord(text: string): string {
    let words: Array<string> = splitIntoWords(text);
    let frequencies: Map<string,number> = countOccurrences(words);
    let winner: string = findMostFrequent(frequencies);
    return winner;
}
```
MostCommonWord() 中的数据流如图所示。
![](/ref/lect9/2023-07-22-13-26-16.png)

假设我们在 countOccurrences() 中遇到了意外异常。这就是我们要调查的故障。然后，我们可以排除该点下游的所有内容（即数据流中的所有内容），将其作为可能出现错误的位置。例如，我们不会通过查找 findMostFrequent() 找到错误，因为故障发生时它还没有被执行。

下面是一些与我们目前掌握的信息相一致的假设。它们按故障发生的时间倒序排列：

* 错误出在 countOccurrences 中：它的输入是有效的，但却抛出了异常
* 错误出在 splitIntoWords 和 countOccurrences 之间的连接上：两个方法都符合它们的契约，但前者保证的后置条件并不满足后者期望的前置条件
* 错误出在 splitIntoWords 中：它的输入是有效的，但却产生了错误的输出
* 错误出在 mostCommonWord 的原始输入中：文本不满足整个方法的先决条件

先尝试哪个假设？Debugging是一个搜索过程，您可以使用Binary search 搜索来加快这一过程。要进行Binary search搜索，你可以将这个数据流分成两半，也许猜测错误出在第一个辅助方法和第二个辅助方法之间的连接上，然后使用下面的experiment之一（如打印语句、断点或断言）来检查那里的值。根据该experiment的答案，您就可以知道是在数据流的前半部分还是后半部分查找错误。


#### Delta debugging

如果在隔离一个小测试用例的过程中发现了两个密切相关的测试用例，而这两个测试用例中一个成功、一个失败，那么这个过程也可能提供有助于形成假设的数据。例如，也许 mostCommonWords("c c, b") 出错了，但 mostCommonWords("c c b") 却正常。现在，您可以检查这两个测试用例执行过程中的差异，以帮助构建您的hypothesis。哪些代码在通过的测试用例中被执行，但在失败的测试用例中被跳过，反之亦然？一种hypothesis是，bug就出在这些代码行中，即通过运行和失败运行之间的差值。

这是错误查找中一般想法的一个具体例子，这种想法被称为三角调试[delta debugging](https://en.wikipedia.org/wiki/Delta_Debugging)，即比较成功运行与失败运行，尝试找出错误所在。当回归测试开始失败时，另一种 delta debug也很有用。使用版本控制系统，您可以检索到仍能通过测试的最新版本，然后系统地探索旧工作版本和新失败版本之间的代码变更，直到找到引入错误的变更。Delta debugging tools 可以自动完成这种搜索过程，不过和切片工具一样，它们还没有得到广泛应用。

#### Prioritize hypotheses

在进行假设时，您可能需要记住系统的不同部分出现故障的可能性不同。例如，经过良好测试的旧代码可能比最近添加的代码更值得信赖。TypeScript 库代码可能比你的代码更值得信赖。TypeScript 编译器和运行时、操作系统平台以及硬件越来越值得信赖，因为它们经过了更多的尝试和测试。在找到不信任的充分理由之前，您应该信任这些较低级别的代码。

### 3. Experiment
你的hypothesis应该导致一个预测，比如 "我认为变量 x 此时的值是错误的"，甚至 "我认为这段代码永远不会被执行"。你应该选择实验来测试预测。最好的实验是  a probe，即对系统进行温和的观察，尽可能少地干扰系统。

#### Assertions, revisited
我们已经详细讨论过一种探测方法：测试变量值或其他内部状态的断言。在上面的例子中，x 永远不允许超过 100，我们可以在任何时候插入 assert(x <= 100); 作为probe。断言的优点是不需要手动检查输出结果，如果测试结果普遍为真，甚至可以在调试后保留在代码中（有些调试断言只对调试的特定测试用例为真，这些断言需要删除）。断言的缺点是，在许多语言（如 Java）中，它们不是默认开启的，因此你可能会被看似通过但实际上根本没有运行的断言所欺骗。

#### Print statements & logging
另一个熟悉的probe是打印语句。Print debugging（由于[历史原因](https://stackoverflow.com/questions/189562/what-is-the-proper-name-for-doing-debugging-by-adding-print-statements)也称为 printf 调试）的优点是几乎适用于所有编程语言。它的缺点是会对程序进行修改，而你必须记住将其还原。在长时间调试后，程序很容易被打印语句弄得乱七八糟。另外，在编写调试打印语句时也要考虑周全。与其在 15 个不同的地方打印相同的 hi！来跟踪程序的执行情况，而弄不清哪个是哪个，不如打印一些清晰的描述性语句，如 calculateTotalBonus 的开始。

logging是一种更复杂的打印调试，它将信息打印语句永久保存在代码中，并通过全局设置（如boolean  DEBUG 常量或log level variable.）来开启或关闭它们。日志级别想法的一个简单版本出现在标准 [JavaScript 控制台 API ](https://developer.mozilla.org/en-US/docs/Web/API/Console#outputting_text_to_the_console)中，它不仅提供了 console.log()，还提供了 console.info()、console.warn() 和 console.error()，代表了所显示消息的重要程度越来越高。控制台框架允许用户过滤日志输出，以隐藏不重要的信息，而发送到最重要日志级别（如 console.error()）的信息通常会以红色显示，因此无论过滤与否，这些信息都会非常显眼。像 [winston ](https://www.npmjs.com/package/winston)这样更复杂的日志框架还可以将日志直接发送到文件或网络服务器上，既可以记录结构化数据，也可以记录人类可读字符串，而且不仅可以用于开发，还可以用于部署。如果没有日志记录，大型复杂系统将很难运行。


与断言一样，您必须注意不要在打印或日志探针中引入side effects。例如，请考虑以下函数：

```ts
function recursiveHelper(arr: Array<number>): number {
  // base case 
  ...
  return recursiveHelper(arr);
}
```
Instead of print debugging like so:
```ts

function recursiveHelper(arr: Array<number>): number {
  // base case 
  ...
  console.log("recursiveHelper returning:", recursiveHelper(arr)); // BAD BAD BAD
  return recursiveHelper(arr);
}
```
It’s much safer to pull the return value into a constant which can be printed and returned:
```ts

function recursiveHelper(arr: Array<number>): number {
  // base case 
  ...
  const output = recursiveHelper(arr);
  console.log("recursiveHelper returning:", output);
  return output;
}
```
将输出保存到常量中可以防止意外的副作用，并避免重复执行相同的代码而增加运行时间，同时确保记录的值确实是返回的值。

最后，谈谈 TypeScript/JavaScript 中的打印调试。一些内置类型（包括 Map 和 Set）的 toString() 方法非常不实用。这意味着，如果您尝试使用 console.log（"map is " + map）进行调试，很可能会看到类似 "map is [object Map]"这样的无用信息。有两种方法可以改善这种情况：

对 console.log() 使用多个参数，而不是依赖 + 和 toString()：
```ts
console.log（"map is", map）；
```
这意味着，console.log() 负责显示map对象，并能更好地向你展示对象中的内容。

使用 util.inspect() 将对象转换为字符串：

import util from 'util'；
console.log("map is " + util.inspect(map))；

#### The debugger

第三种probe方法是使用调试器设置断点，在断点处停止程序运行，允许你单步浏览程序并检查变量值。debugger是一个功能强大的工具，只要努力学习如何使用它，就会有所收获。

接下来，要学习如何设置断点、移除断点和启动调试会话.

调试器提供了窥视系统幕后的功能，允许你观察程序在执行过程中的状态，并对其演变进行精细控制。大多数调试器提供五步操作：

* 步入（Step in），在暂停行的第一个函数调用中迈出一步
* 退出，完成被调用者的执行，并在被调用者返回后立即暂停回到调用者处
* 继续（Continue），继续执行程序，直到另一个断点被击中或程序执行完毕
* Restart 重新启动调试会话
* Stop 断开调试会话

停止时，调试器提供了一种方便的方法，可通过调试控制台（也称为调试 REPL）检查暂停的程序。(您可能还记得 6.101 中的 LISP 实验室中的缩写 REPL，即读取-执行-打印循环）。您可以在程序的当前（暂停）帧中使用调试控制台执行语句，检查局部变量和函数定义等内容。

Debuggers有时可以在程序执行过程中更改部分内容，如变量值。不建议在运行时修改值，这往往会导致更多混乱。

Print debugging 的强大之处在于它简单易用--大多数程序员学会的第一件事就是打印！但是，Print debugging 也要求你提前决定在哪里、探查什么，并在事后拼凑出发生了什么。使用debugger可以让你自由地提出后续问题，并在出现错误的程序中进行深入探讨，是更快发现错误的有效方法。调试器的另一个好处是使用后无需清理。

值得注意的是，debugger并非 VS Code 所独有。最著名的调试器之一 gdb 于 1986 年首次发布，至今仍被广泛使用（不过，由于缺乏本地图形用户界面，gdb 也是最难学的调试器之一）。

debugger（顾名思义）也是调试以外的有用工具。调试器对程序理解也很有帮助--探索和理解一个庞大的代码库，虽然这个代码库不是你写的，但你必须对其进行修改。

还有一句忠告：
> “Even when working with an interactive debugger, one should always be explicit about the current hypothesis, and the steps required to validate it… Debuggers can be excellent tools—but only when combined with good thinking.”
> Zeller, Why Programs Fail, Chapter 8

#### One bug at a time
当你试图调试一个问题时，发现其他问题是很正常的。也许你注意到其他模块返回了错误的答案。也许当你阅读自己的代码时（实际上是自我代码审查），你会发现一些明显的错误需要修正。

Keep a bug list。这有助于你处理调试过程中出现的新问题。在bug list 中写下这些问题，以便日后再处理。bug list 可以是简单的纸质笔记本或文本文件。对于小组软件开发，最好使用在线错误跟踪器，如 GitHub 的 Issues 标签。

不要从正在处理的错误中分心。反思一下新问题是否为您正在研究的错误提供了信息数据--它是否带来了新的假设？但不要立即开始对新的错误进行递归调试，因为你可能很难跳出思维堆栈，回到原来的错误上。也不要在调试时随意修改代码，因为你不知道这些修改是否会影响你的调试实验。请将代码修改的重点放在每次对一个错误进行仔细、可控的探测上。

如果新问题干扰了你的调试能力--例如，程序间歇性崩溃，导致你无法可靠地运行experiments--那么你可能需要重新安排修复错误的优先次序。放下当前的错误（解开或注释您的实验探针，并确保该错误在您的错误列表中，以便稍后修复），转而处理新的错误。

#### Don’t fix yet
尝试做一个似乎能修复假设错误的实验，而不是仅仅做一个probe，这很有诱惑力。这几乎总是错误的做法。首先，它会导致一种临时性的猜测和测试编程，从而产生可怕、复杂、难以理解的代码。其次，你的修复可能只是掩盖了真正的错误，而没有真正消除它--治标不治本。

例如，如果您遇到 RangeError，不要仅仅添加代码捕获异常并忽略它。请确保您已经理解了异常抛出的原因，并修复了导致异常的真正问题。

### 4. Repeat

实验结束后，思考实验结果并修改假设。如果你观察到的结果与假设预测的不一致，那么就否定该假设，并提出一个新的假设。如果观察结果与预测一致，则完善假设，缩小错误位置和原因的范围。然后用这个新的假设重复上述过程。

Keep an audit trail
这是 one of Agans’ nine rules of debugging: Keep an Audit Trail。

如果发现一个错误需要几分钟以上的时间，或者在 "研究-假设-实验 "的循环中反复进行几次以上，那么你就需要开始把事情写下来，因为你大脑中的短期记忆很快就会忘记哪些是有效的，哪些是无效的。

在文本文件中记录下你所做的事情、顺序以及结果。包括

* 你现在正在探索的假设
* 你现在尝试测试该假设的实验
* 你观察到的实验结果：
  * 这次测试是通过了还是失败了
  * 程序输出，尤其是你自己的调试信息
  * 任何堆栈跟踪
系统的调试技术会迅速超出你大脑的记忆能力，所以尽早开始写下来吧。最小化测试用例和 delta debugging 通常需要多次反复才能找到答案。如果你不写下你正在尝试哪个测试用例，以及它是通过了还是失败了，那么你就很难跟踪你正在做什么。

#### Check the plug
如果您一直在迭代一个错误，但没有任何意义，那么请考虑阿甘斯的另一条规则：Check the plug。这意味着你应该质疑自己的假设。例如，如果你打开了机器的电源开关，但机器却没有启动，也许你不应该调试开关或机器，而应该问机器是否插上了电源？或者插座本身是否有电？

在编程中，"Check the plug"的一个例子就是确保源代码和目标代码都是最新的。如果你的观察结果都不合理，一个可能的假设是你正在运行的代码（目标代码）与你正在阅读的代码（源代码）不匹配。要测试这一点，可以从版本库中提取最新版本，删除所有 JavaScript 输出文件（以 .js 结尾的文件），然后重新编译所有 TypeScript 代码。

Six Stages of Debugging:
1. That can’t happen.
2. That doesn’t happen on my machine.
3. That shouldn’t happen.
4. Why does that happen?
5. Oh, I see.
6. How did that ever work?
Zeller, Why Programs Fail, Chapter 2

If YOU didn’t fix it, it isn’t really fixed
这是阿甘斯九条规则中的另一条。在一个复杂的系统中，有时一个错误会突然消失。也许是你做了什么，但也许不是。如果系统现在看起来正常运行，而你却不知道错误是如何被修复的......那么很可能它并没有被修复。错误只是又躲了起来，被环境中的一些变化所掩盖，但随时都有可能再次出现。在本课程的后半部分，当我们讨论并发性时，我们会看到一些类似这种令人讨厌的错误的例子。

系统调试有助于建立信心，相信错误是真的被修复了，而不仅仅是暂时被隐藏了。这就是为什么要首先重现错误，以便看到系统处于失效状态。然后，在尚未修复错误的情况下了解错误的原因。最后，利用您对原因的了解，应用变更来修复错误。为了确信自己真的修复了错误，您需要看到您的更改导致系统从失效状态转变为正常工作状态，并了解原因。

## Fix the bug

找到错误并了解其原因后，第三步就是设计修复方案。要避免打一个补丁就了事的诱惑。问问自己，这个错误是编码错误（如拼写错误的变量或互换的方法参数），还是设计错误（an underspecified or insufficient interface）。设计错误可能会建议你后退一步，重新审视你的设计，或者至少考虑一下失效接口的所有其他客户端，看看它们是否也存在这个错误。

查找相关错误和新出现的错误。想想代码的其他地方是否也会出现类似的错误。如果我刚刚在这里发现了除以零的错误，那么我是否在代码的其他地方也出现过这样的错误？尽量使代码避免今后再出现类似错误。还要考虑您的修正会产生什么影响。是否会破坏其他代码？

Undo debugging probes。在发现 bug 的过程中，你可能注释了代码、添加了打印语句或做了其他改动，以探测程序的行为或使调试更快。在将修复提交到版本控制之前，请立即撤销所有这些改动。查看 git diff 的输出是确保你已经删除了所有probe。

进行回归测试。应用修复后，将漏洞测试用例添加到回归测试套件中，并运行所有测试以确保 (a) 漏洞已修复，以及 (b) 没有引入新的漏洞。

Add, Commit, Push, and Reflect。通过Add, Commit, Push, and Reflect保存工作代码。然后进行反思。最有效的调试策略是什么？今后如何防止类似情况发生？You should now feel a new sense of ownership and have an even deeper grasp over your system。如果没有，这可能表明你只是暂时修补了漏洞的症状，而不是它的原因。

### Other tips
Get a fresh view。
这是阿甘斯的九条调试规则中的另一条。向别人解释你的问题通常很有帮助，即使对方根本不知道你在说什么。这有时被称为[rubber-duck debugging](https://en.wikipedia.org/wiki/Rubber_duck_debugging)或teddy-bear debugging。有一个计算机实验室在服务台旁边放了一只大泰迪熊，规定你必须先 "告诉泰迪熊"，然后再尝试人类。泰迪熊独自处理的问题数量惊人。大声说出你期望代码工作的原因，以及代码在做什么，可以很好地帮助你自己意识到问题所在。

Describe it in writing。
如果泰迪熊帮不上忙，可以尝试写一封（假装的）电子邮件寻求帮助。大多数人没有时间阅读关于你的错误的论文，但如果没有足够的细节或努力的证明，大多数人也不会愿意提供帮助。以您的审计线索为指导，将自己的内容限制在 1-2 段，在不牺牲重要细节的前提下尽量简洁。

首先描述症状。您为最小化错误所付出的努力将帮助您制作一个[最小的、可重现的示例](https://stackoverflow.com/help/minimal-reproducible-example)（您也可以用它来在线提问）。预期行为和实际行为是什么？如果出现运行时错误，您能从中获取哪些信息？试着将您的程序想象成一个物理系统；描述系统中哪里出了问题。

然后描述您的实验。您已经测试了哪些假设？你查看了 StackOverflow 上的哪些帖子？为什么它们不足以解决错误？

最后，重读并修改。问问自己 如果我从别人那里收到这份错误报告，我会觉得有能力帮助他们吗？你是否可以加强任何描述？你是否真的尝试了所有可用的调试策略并穷尽了所有假设？在以书面形式正式描述错误的过程中，您很可能会获得新的见解，发现新的假设进行测试，甚至[解决错误](https://blog.regehr.org/archives/208)。

你简明扼要的问题报告甚至可能会在向课程工作人员寻求帮助时派上用场（毕竟，在实验时间内，时间可能是至关重要的！），而且肯定会成为今后撰写错误报告或向同事寻求帮助时的良好练习。

寻求帮助。
编程很难，错误在所难免。6.102 教员和同学通常知道你在说什么，所以他们更适合寻求帮助。在课外，你可以向其他团队成员、同事或 StackOverflow 寻求新的意见。

睡一觉。
如果你太累了，你就不会是一个有效的调试者。Trade latency for efficiency.。

## Summary
In this reading, we looked at some ways to minimize the cost of debugging, and how to debug systematically:

* Avoid debugging
  * make bugs impossible with techniques like static typing, automatic dynamic checking, and immutability
* Keep bugs confined
  * failing fast with assertions keeps a bug’s effects from spreading
  * incremental development and unit testing confine bugs to your recent code
    * add, commit, and push working code often!
  * modularity, encapsulation, and scope minimization reduce the amount of the program you have to search
* Debug systematically
  * reproduce the bug as a test case, and put it in your regression suite
  * find the bug using the scientific method:
    * generate hypotheses using binary search and delta debugging
    * use minimially-invasive probes, like print statements, assertions, or a debugger, to observe program behavior and test the prediction of your hypotheses
  * fix the bug thoughtfully
Thinking about our three main measures of code quality:

* Safe from bugs. We’re trying to prevent them and get rid of them.
* Easy to understand. Techniques like static typing, const/readonly declarations, and assertions are additional documentation of the assumptions in your code. Variable scope minimization makes it easier for a reader to understand how the variable is used, because there’s less code to look at.
* Ready for change. Assertions and static typing document the assumptions in an automatically-checkable way, so that when a future programmer changes the code, accidental violations of those assumptions are detected.

Line one
    Line two
    Line three
    Line four

```ts
var foo = 1;
var bar = 'a';
var foobar = foo + bar;
```

```ts

var foo = "method(" + argument1 + "," + argument2 + ")";
```
...We're waiting for copy before the site can go live...
...If you are content with this, let's go ahead with it...
...We'll launch as soon as we have the copy...

<a href="#">ONE</a>
<a href="#">TWO</a>
<a href="#">THREE</a>

```ts
var foo = 1;
var bar = 'a';
var foobar = foo + bar;
```